In [ ]:
import sys
import traceback
import ipywidgets as widgets
import itertools
import networkx as nx
from collections import defaultdict
from typing import Dict, List, Tuple

import ipywidgets as widgets
from IPython.display import display

import ipywidgets as widgets

intro_text = widgets.HTML(
    value="""
    <div style="font-family: Arial, sans-serif; line-height: 1.6; padding: 10px; border: 1px solid #ccc; border-radius: 10px; background-color: #f9f9f9;">
        <h2 style="margin-top: 0;"> SWEMLS Visualizer</h2>
        <p>This tool enables the visualization of metadata and workflows of hybrid AI systems described in the Semantic Web and Machine Leanring Systems Ontology (Ekaputra et al. 2023) and generates draw.io-compatible XML files.</p>
        <p>🔗 You can find more information about the SWEMLS Ontology here:<br>
            <a href="https://semsys.ai.wu.ac.at/sites/swemls-kg/" target="_blank">
                https://semsys.ai.wu.ac.at/sites/swemls-kg/
            </a>
        </p>

        <h4 style="margin-top: 20px;"> User Guide</h4>
        <ul style="padding-left: 20px;">
            <li><strong> Select your data source:</strong><br>
                Use the provided example file (containing 400+ systems) or upload your own TTL file.<br>
                Click <em>Continue</em> to proceed.
            </li>
            <li><strong> Search for a system:</strong><br>
                Use a SPARQL query to find a matching system.<br>
                Confirm your selection by clicking <em>Confirm Selection</em>.<br>
                The default query searches for systems that match specific patterns (e.g., A1).
            </li>
            <li><strong> Choose a visualization method:</strong><br>
                If the pattern is known, choose one of the following options:
                <ul style="padding-left: 20px;">
                    <li> Use a manually created template</li>
                    <li> Automatically generate the workflow using the layout algorithm</li>
                </ul>
                Click <em>Continue with Selected Option</em> to proceed.<br>
                To reset your selection, simply click <em>Confirm Selection</em> (selecting the same system) again.
            </li>
            <li><strong> If the pattern is unknown:</strong><br>
                Please upload a valid pattern TTL file and click <em>Continue with Uploaded Pattern</em>.
            </li>
            <li><strong> Download:</strong><br>
                Click the generated download link to obtain the XML file.<br>
                You can import this file into draw.io to view the visualization.
            </li>
        </ul>
        <hr style="margin-top: 20px;">
    </div>
    """,
    layout=widgets.Layout(margin='0 0 20px 0')
)


display(intro_text)


# Definiere das Output-Widget
error_output = widgets.Output()
display(error_output)

import xml.etree.ElementTree as ET

METADATA_TEMPLATE_PATH = "Templates/MetaTemp.xml"
MERGED_OUTPUT_PATH = "Merged_Workflow_Output.xml"

def finalize_and_merge_xml(xml_string_or_path):
    with download_output:
        download_output.clear_output()
        try:
            with open(METADATA_TEMPLATE_PATH, "r", encoding="utf-8") as f:
                meta_xml_template = f.read()

            updated_meta_xml = replace_paper_placeholders(meta_xml_template, instance_data)
            meta_output_path = "Updated_Metadata.xml"

            with open(meta_output_path, "w", encoding="utf-8") as f:
                f.write(updated_meta_xml)

            if isinstance(xml_string_or_path, str) and not os.path.isfile(xml_string_or_path):
                workflow_xml_path = "Generated_Workflow.drawio"
                with open(workflow_xml_path, "w", encoding="utf-8") as f:
                    f.write(xml_string_or_path)
            else:
                workflow_xml_path = xml_string_or_path

            # 🛡️ Prüfung vor Merge
            #print(f"🔍 Merge-Vorbereitung:")
            #print(f"  → Metadata XML: {meta_output_path}, Größe: {os.path.getsize(meta_output_path)} Bytes")
            #print(f"  → Workflow XML: {workflow_xml_path}, Größe: {os.path.getsize(workflow_xml_path)} Bytes")

            with open(workflow_xml_path, "r", encoding="utf-8") as f:
                first_line = f.readline()
                #print(f"  → Erste Zeile der Workflow-XML: {first_line.strip()}")

            if os.path.getsize(workflow_xml_path) == 0:
                #print(f"❌ Merge abgebrochen: Workflow-XML '{workflow_xml_path}' ist leer.")
                return

            merge_drawio_xml(meta_output_path, workflow_xml_path, MERGED_OUTPUT_PATH)

            b64 = base64.b64encode(open(MERGED_OUTPUT_PATH, "rb").read()).decode()
            href = f'data:application/xml;base64,{b64}'
            display(HTML(f'<a download="{MERGED_OUTPUT_PATH}" href="{href}" target="_blank">⬇️ Click here to download the XML</a>'))

        except Exception as e:
            show_exception(e)



def style_component_cells(cells, stroke_width=2.5):
    #print("[INFO] Setze dicke Rahmen für Komponenten mit shape=rectangle oder shape=hexagon")
    for cell in cells:
        style = cell.attrib.get("style", "")
        if "shape=rectangle" in style or "shape=hexagon" in style:
            # strokeWidth setzen oder ersetzen
            if "strokeWidth=" in style:
                style = re.sub(r'strokeWidth=\d+(\.\d+)?', f'strokeWidth={stroke_width}', style)
            else:
                if not style.endswith(";") and style != "":
                    style += ";"
                style += f"strokeWidth={stroke_width};"

            # strokeColor NICHT setzen!
            cell.attrib["style"] = style


def load_root_cells(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    diagram_root = root.find('.//root')
    cells = diagram_root.findall('mxCell')
    return tree, root, cells

def get_min_y(cells):
    return min(
        (float(cell.find("mxGeometry").attrib.get("y", "0"))
         for cell in cells if cell.find("mxGeometry") is not None),
        default=0.0
    )

def get_min_x(cells):
    return min(
        (float(cell.find("mxGeometry").attrib.get("x", "0"))
         for cell in cells if cell.find("mxGeometry") is not None),
        default=0.0
    )

def get_left_x_by_cell_id(cells, target_id):
    for cell in cells:
        if cell.get("id") == target_id:
            geom = cell.find("mxGeometry")
            if geom is not None:
                return float(geom.attrib.get("x", "0"))
    return 0.0

def apply_y_offset(cells, offset_y=0):
    #print(f"[INFO] Verschiebe Workflow vertikal um y = {offset_y}")
    for cell in cells:
        geom = cell.find("mxGeometry")
        if geom is not None:
            y = float(geom.attrib.get("y", "0"))
            geom.attrib["y"] = str(y + offset_y)

def apply_x_offset(cells, offset_x=0):
    #print(f"[INFO] Verschiebe Workflow horizontal um x = {offset_x}")
    for cell in cells:
        geom = cell.find("mxGeometry")
        if geom is not None:
            x = float(geom.attrib.get("x", "0"))
            geom.attrib["x"] = str(x + offset_x)

def shift_workflow_nodes_below_metadata(root, table_ids, padding=50):
    max_y = 0
    for table_id in table_ids:
        table_cell = root.find(f".//mxCell[@id='{table_id}']")
        if table_cell is not None:
            geom = table_cell.find("mxGeometry")
            if geom is not None:
                try:
                    y = float(geom.get("y", "0"))
                    h = float(geom.get("height", "0"))
                    max_y = max(max_y, y + h)
                except ValueError:
                    continue
    return max_y + padding

def merge_drawio_xml(xml1_path, xml2_path, output_path):
    tree1, root1, cells1 = load_root_cells(xml1_path)
    _, root2, cells2 = load_root_cells(xml2_path)

    table_ids = [
        "dXy3HHlULE7lc8IMxovX-26", #sys
        "dXy3HHlULE7lc8IMxovX-1",  #paper
        "dXy3HHlULE7lc8IMxovX-48"   # doc
    ]

    workflow_cells = [
        c for c in cells2
        if c.get("vertex") == "1" and c.get("parent") not in table_ids
    ]

    #print("=== DEBUG: Vorher - Workflow-Knotenpositionen ===")
    for cell in workflow_cells:
        geom = cell.find("mxGeometry")
        #if geom is not None:
            #print(f"  ↳ ID: {cell.get('id')}, x: {geom.get('x', '?')}, y: {geom.get('y', '?')}")

    offset_y = shift_workflow_nodes_below_metadata(root1, table_ids, padding=50)
    min_workflow_y = get_min_y(workflow_cells)
    apply_y_offset(workflow_cells, offset_y - min_workflow_y)

    system_x = get_left_x_by_cell_id(cells1, "dXy3HHlULE7lc8IMxovX-26")
    min_x = get_min_x(workflow_cells)
    apply_x_offset(workflow_cells, system_x - min_x)
    style_component_cells(workflow_cells, stroke_width=2.5)

    #print("=== DEBUG: Nachher - Workflow-Knotenpositionen ===")
    for cell in workflow_cells:
        geom = cell.find("mxGeometry")
        #if geom is not None:
           # print(f"  ↳ ID: {cell.get('id')}, x: {geom.get('x', '?')}, y: {geom.get('y', '?')}")

    # Merging beide root-Blöcke
    merged_root = ET.Element("root")
    seen_ids = set()
    base_ids = {"0", "1"}

    for cell in cells1 + cells2:
        cid = cell.get("id")
        if cid in base_ids:
            if cid not in seen_ids:
                merged_root.append(cell)
                seen_ids.add(cid)
        else:
            merged_root.append(cell)

    graph_model = tree1.find(".//mxGraphModel")
    old_root = graph_model.find("root")
    graph_model.remove(old_root)
    graph_model.append(merged_root)

    tree1.write(output_path, encoding="utf-8", xml_declaration=True)
    #print(f"✅ Merged XML saved to: {output_path}")

# ✅ Finaler Check: node_aliases, all_nodes, label_map
def replace_paper_placeholders(xml_string, instance_data):
    tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()

    paper = instance_data["papers"][0]
    paper_meta = paper["metadata"]
    system = instance_data

    # Ersetzungen vorbereiten
    replacements = {
        "Add.Title": paper_meta.get("has_title", [""])[0],
        "Add.AuthorCountry": paper_meta.get("hasAuthorsCountry", [""])[0].split("#")[-1],
        "Add.Year": paper_meta.get("year", [""])[0],
        "Add.Venue": paper_meta.get("venue", [""])[0],
        "Add.Keyword": ", ".join(paper_meta.get("hasKeyword", [])),
        "Add.ID": paper_meta.get("has_ID", [""])[0],
        "Add.PaperType": paper_meta.get("isOfType", [""])[0].split(".")[-1],
        "Add.Reports": paper_meta.get("reports", [""])[0].split("/")[-1],
        "Add.AplicationDomain": system["relationships"].get("hasApplicationDomain", [""])[0].split(".")[-1],
        "Add.Area": system["relationships"].get("hasArea", [""])[0].split(".")[-1],
        "Add.Pattern": system["relationships"].get("hasCorrespondingPattern", [""])[0].split(".")[-1],
        "Add.StatisticalModels": ", ".join(m.split(".")[-1] for m in system["relationships"].get("hasStatisticalModel", [])),
        "Add.SystemMaturity": system["relationships"].get("hasSystemMaturity", [""])[0].split(".")[-1],
        "Add.Task": system["relationships"].get("hasTask", [""])[0].split(".")[-1],
        "Add.TrainingType": system["relationships"].get("hasTrainingType", [""])[0].split(".")[-1],
        "Add.EvaluationData": system["documentation"].get("evaluation_data", ""),
        "Add.EvaluationDataSplit": system["documentation"].get("evaluation_data_split", ""),
        "Add.EvaluationMetrics": system["documentation"].get("evaluation_metrics", ""),
        "Add.Infrastructure": system["documentation"].get("infrastructure", ""),
        "Add.Parameters": system["documentation"].get("parameters", ""),
        "Add.ProcessSteps": system["documentation"].get("process_steps", ""),
        "Add.ProvenanceSupport": system["documentation"].get("provenance_support", ""),
        "Add.Software": system["documentation"].get("software", ""),
        "Add.SystemID": system["id"]
    }

    for element in root.iter("mxCell"):
        if 'value' in element.attrib:
            value = element.attrib['value']
            for placeholder, real_value in replacements.items():
                if placeholder in value:
                    replacement = real_value.replace("_", " ") if real_value and str(real_value).strip() else "Missing"
                    element.attrib['value'] = value.replace(placeholder, replacement)
                    
                    
                    # system überschirft soll auch fett sein
                    if placeholder == "Add.SystemID":
                        current_style = element.attrib.get('style', '')
                        if "fontStyle=1" not in current_style:
                            if "fontStyle=" in current_style:
                                current_style = re.sub(r'fontStyle=\d+', 'fontStyle=1', current_style)
                            else:
                                if not current_style.endswith(';') and current_style != '':
                                    current_style += ';'
                                current_style += 'fontStyle=1;'
                            element.attrib['style'] = current_style

    return ET.tostring(root, encoding='utf8', method='xml').decode()


def generate_drawio_xml(positions, components, edges, label_map=None):
    #print("\n🔹 FUNCTION CALLED: generate_drawio_xml")

    mxfile = ET.Element("mxfile", host="Electron", version="26.0.16")
    diagram = ET.SubElement(mxfile, "diagram", name="Generated Workflow")
    mxGraphModel = ET.SubElement(diagram, "mxGraphModel", dx="913", dy="789", grid="0", gridSize="10", 
                                 guides="1", tooltips="1", connect="1", arrows="1", fold="1", page="1", 
                                 pageScale="1", pageWidth="827", pageHeight="1169", background="none", 
                                 math="0", shadow="0")
    root = ET.SubElement(mxGraphModel, "root")

    # Draw.io root cells
    ET.SubElement(root, "mxCell", id="0")
    ET.SubElement(root, "mxCell", id="1", parent="0")

    styles = {
        "data": "shape=hexagon;fillColor=#CCCCCC;strokeColor=#999999;",
        "symbolic_data": "shape=hexagon;fillColor=#B9E0A5;strokeColor=#009900;",
        "ml_component": "shape=rectangle;fillColor=#00FFFF;strokeColor=#66B2FF;",
        "sr_component": "shape=rectangle;fillColor=#00FF00;strokeColor=#008000;",
        "kr_component": "shape=rectangle;fillColor=#00FF00;strokeColor=#008000;",
        "unclassified": "shape=ellipse;fillColor=#FFA07A;strokeColor=#FF4500;"
    }

    node_elements = {}

    #print("\n📌 DEBUG: Components received in function:")
    #for category, nodes in components.items():
        #print(f"  - {category}: {nodes}")

    for category, nodes in components.items():
        for node in nodes:
            style = styles.get(category, "shape=ellipse;")

            # 🆕 Neu: Wenn label_map vorhanden und Node bekannt ist, nutze das lesbare Label
            if label_map and node in label_map:
                cleaned_name = label_map[node]
            else:
                cleaned_name = node.replace("Custom.", "").replace("Resource.", "")

            length = len(cleaned_name)
            if length > 24:
                style += "fontSize=5"
            elif length > 20:
                style += "fontSize=7;"
            elif length > 17:
                style += "fontSize=9;"

            pos = positions.get(node, (100, 100))
            mxCell = ET.SubElement(root, "mxCell", id=node, value=cleaned_name, style=style, vertex="1", parent="1")
            mxGeometry = ET.SubElement(mxCell, "mxGeometry", x=str(pos[0]), y=str(pos[1]), width="120", height="80")
            mxGeometry.set("as", "geometry")

            node_elements[node] = node

    #print("\n📌 DEBUG: Checking node elements before edge creation:", node_elements.keys())
    #print("\n📌 DEBUG: Edges list inside function (before alias replacement):", edges)

    updated_edges = []
    for source, target in edges:
        new_source = node_aliases.get(source, source)
        new_target = node_aliases.get(target, target)
        updated_edges.append((new_source, new_target))

    edges = updated_edges

    #print("\n📌 DEBUG: Reached edge processing loop...")

    for source, target in edges:
        #print(f"  🔍 DEBUG: Processing edge {source} → {target}")
        if source in node_elements and target in node_elements:
            #print(f"  ✅ Creating edge: {source} → {target}")
            edge_id = f"edge_{source}_{target}"

            # 🧠 Improved orthogonal edge style for smooth top-down flow
            style = (
                "edgeStyle=straight;"
                "curved=0;"
                "rounden=1;"
                "html=1;"
                "exitX=0.5;"
                "exitY=1;"
                "exitDx=0;"
                "exitDy=0;"
                "entryX=0.5;"
                "entryY=0;"
                "entryDx=0;"
                "entryDy=0;"
            )

            mxCell = ET.SubElement(root, "mxCell", 
                id=edge_id,
                source=source,
                target=target,
                style=style,
                edge="1", parent="1")
            mxGeometry = ET.SubElement(mxCell, "mxGeometry", relative="1")
            mxGeometry.set("as", "geometry")
        else:
            print(f"  ❌ DEBUG: Edge skipped (missing node): {source} → {target}")

    return ET.tostring(mxfile, encoding='utf8', method='xml').decode()

def finalize_layout_parameters():
    global node_aliases, label_map

    #print("\n🧩 Starte finalen Parameter-Check für Komponenten, Variablen und Aliase...")

    # ✅ Dynamically create node_aliases from instance_data
    node_aliases = {}

    for step in instance_data["steps"]:
        short_id = step["id"].split(".")[-1]
        node_aliases[short_id] = step["id"]
        node_aliases[step["id"]] = step["id"]

    for var_key, var_data in instance_data["variables"].items():
        node_aliases[var_key] = var_data["id"]
        node_aliases[var_data["id"]] = var_data["id"]

    # Falls vorhanden, unclassified bereinigen
    if "unclassified" in components:
        components["unclassified"] = [
            node for node in components["unclassified"]
            if node not in node_aliases and node not in node_aliases.values()
        ]
        print("\n🚀 DEBUG: Cleaned 'unclassified' category to remove duplicates:", components["unclassified"])

    # ✅ Labels erstellen
    label_map = {}
    for step in instance_data["steps"]:
        full_id = step["id"]
        short_label = extract_label_short(step)
        label_map[full_id] = short_label

    #print("\n🧾 Final Label Map:")
    #for node_id, label in label_map.items():
        #print(f"  {node_id} → {label}")

    # ✅ XML erzeugen
    #print("\n🧩 Starte finale XML-Erzeugung mit generiertem Layout...")
    xml_output = generate_drawio_xml(positions, components, edges, label_map)

    # ✅ Download-Link erzeugen
    instance_id = instance_data.get("id", "unknown")
    filename = f"Generated_{instance_id}_Workflow.drawio"
    b64 = base64.b64encode(xml_output.encode()).decode()
    href = f'data:application/xml;base64,{b64}'

    #display(HTML(f'<a download="{filename}" href="{href}" target="_blank">⬇️ Click here to download the generated XML</a>'))
    #print(f"✅ File ready to Download: {filename}")

# Funktion zur Fehlerausgabe
def show_exception(e):
    with error_output:
        error_output.clear_output()  # Lösche vorherige Ausgaben
        print("❌ Exception caught:")
        traceback.print_exception(type(e), e, e.__traceback__, file=sys.stdout)

def shift_with_cascade(pos_to_node, positions, x, y, shift_x):
    """
    Versucht, (x, y) zu belegen – wenn dort schon ein Node ist, wird er nach rechts verschoben,
    inkl. Kaskade für weitere Blockaden.
    """
    if (x, y) not in pos_to_node:
        return x  # Kein Problem – Platz ist frei

    # Es gibt bereits einen Node hier → verschiebe ihn nach rechts
    blocked_node = pos_to_node[(x, y)]
    new_x = shift_with_cascade(pos_to_node, positions, x + shift_x, y, shift_x)

    #print(f"  🔁 Kollision: {blocked_node} ist auf ({x}, {y}) → verschiebe nach ({new_x}, {y})")
    positions[blocked_node] = (new_x, y)
    pos_to_node[(new_x, y)] = blocked_node
    del pos_to_node[(x, y)]  # alte Position freigeben

    return x  # Jetzt kann der ursprüngliche Node an diese Stelle

def simple_overlap_correction(positions, edges, x_spacing=200, shift_x=200):
    #print("\n🧠 Starte finalen Überlappungs-Check mit Shift-Kaskade...")

    pos_to_node = {(x, y): node for node, (x, y) in positions.items()}
    x_to_y = defaultdict(list)
    for (x, y), node in pos_to_node.items():
        x_to_y[x].append((y, node))

    corrections = {}

    for source, target in edges:
        if source not in positions or target not in positions:
            continue

        x1, y1 = positions[source]
        x2, y2 = positions[target]

        if x1 == x2 and abs(y1 - y2) > 1:
            y_min, y_max = sorted([y1, y2])
            for y_mid, middle_node in x_to_y[x1]:
                if y_min < y_mid < y_max:
                    #print(f"⚠️ Überlappung erkannt bei Kante {source} → {target} durch {middle_node} auf X={x1}, Y={y_mid}")
                    
                    if target not in corrections:
                        # Jetzt: Berechne neue X-Position mit Kaskade
                        new_x = shift_with_cascade(pos_to_node, positions, x1 + shift_x, y2, shift_x)
                        corrections[target] = (new_x, y2)
                    break

    for node, (new_x, new_y) in corrections.items():
        #print(f"➡️ Verschiebe Node {node} nach rechts auf X={new_x}")
        positions[node] = (new_x, new_y)
        pos_to_node[(new_x, new_y)] = node

    #if not corrections:
     #   print("✅ Keine Überlappungen erkannt – keine Verschiebungen nötig.")
    #else:
     #   print("✅ Überlappungs-Korrektur (mit Kaskade) abgeschlossen.")

    return positions


def assign_coordinates(ordered_layers: Dict[int, List[str]], x_spacing: int = 200, y_spacing: int = 150) -> Dict[str, Tuple[int, int]]:
    positions = {}
    for rank in sorted(ordered_layers):
        nodes = ordered_layers[rank]
        for i, node in enumerate(nodes):
            x = i * x_spacing
            y = rank * y_spacing  
            positions[node] = (x, y)
    return positions

# Funktion zur Berechnung von Kantenüberschneidungen
def count_crossings(layers: Dict[int, List[str]], edges: List[Tuple[str, str]]) -> int:
    try:
        pos_in_layer = {node: (rank, i) for rank, nodes in layers.items() for i, node in enumerate(nodes)}
        crossings = 0
        for (u1, v1), (u2, v2) in itertools.combinations(edges, 2):
            if u1 not in pos_in_layer or v1 not in pos_in_layer or u2 not in pos_in_layer or v2 not in pos_in_layer:
                continue
            r1_u, x1_u = pos_in_layer[u1]
            r1_v, x1_v = pos_in_layer[v1]
            r2_u, x2_u = pos_in_layer[u2]
            r2_v, x2_v = pos_in_layer[v2]
            if r1_v != r2_v:
                continue  # Nur Kanten im gleichen Layer vergleichen
            if (x1_u - x2_u) * (x1_v - x2_v) < 0:
                crossings += 1
        return crossings
    except Exception as e:
        show_exception(e)  # Fehlerausgabe aufrufen
        return 0

# Funktion zur gierigen Swap-Optimierung
def greedy_swap_optimization(layers: Dict[int, List[str]], edges: List[Tuple[str, str]], iterations: int = 10) -> Dict[int, List[str]]:
    try:
        G = nx.DiGraph()
        G.add_edges_from(edges)
        new_layers = {r: list(nodes) for r, nodes in layers.items()}  # Kopie der Schichten
        #print("\n🔁 Starte Greedy-Swap Optimierung...\n")
        for r in sorted(new_layers):
            if r == 0:
                continue  # Layer 0 nicht optimieren
            improved = True
            while improved:
                improved = False
                current = new_layers[r]
                for i in range(len(current) - 1):
                    swapped = list(current)
                    swapped[i], swapped[i+1] = swapped[i+1], swapped[i]
                    temp_layers = dict(new_layers)
                    temp_layers[r] = swapped
                    old_crossings = count_crossings(new_layers, edges)
                    new_crossings = count_crossings(temp_layers, edges)
                    if new_crossings < old_crossings:
                        print(f"  ✅ Swap: {current[i]} ⬌ {current[i+1]} reduziert Kreuzungen: {old_crossings} → {new_crossings}")
                        new_layers[r] = swapped
                        improved = True
                        break
        return new_layers
    except Exception as e:
        show_exception(e)  # Fehlerausgabe aufrufen
        return {}

# Funktion zur Anwendung der Median-Heuristik
def apply_median_heuristic(rank_map: Dict[str, int], edges: List[Tuple[str, str]], sort_layer_0: bool = False) -> Dict[int, List[str]]:
    try:
        #print("\n🔧 Starte Median-Heuristik zur Anordnung der Knoten innerhalb von Rängen...\n")
        
        # Prüfen, ob rank_map und edges korrekt sind
        if not rank_map or not edges:
            print("❌ Error: rank_map or edges are empty.")
            return {}
        
        layers = defaultdict(list)
        for node, rank in rank_map.items():
            layers[rank].append(node)

        if not layers:
            print("❌ Error: Layers are empty.")
            return {}

        G = nx.DiGraph()
        G.add_edges_from(edges)
        ordered_layers = {}

        for r in sorted(layers.keys()):
            if r == 0 and not sort_layer_0:
                ordered_layers[r] = layers[r]
                continue
            
            def median(node):
                preds = list(G.predecessors(node))
                if not preds:
                    return float('inf')
                return sum([layers[rank_map[pred]].index(pred) for pred in preds if pred in layers[rank_map[pred]]]) / len(preds)

            sorted_nodes = sorted(layers[r], key=median)
            ordered_layers[r] = sorted_nodes
            
            # Ausgabe der vertauschten Komponenten
            #print(f"\nRang {r} - ursprüngliche Reihenfolge: {layers[r]}")
            #print(f"  Neu angeordnet: {sorted_nodes}")

        #print("🔧 Ergebnis der Median-Heuristik:")
        #for rank in sorted(ordered_layers):
            #print(f"  Rang {rank}: {ordered_layers[rank]}")
        
        return ordered_layers
    except Exception as e:
        show_exception(e)  # Fehlerausgabe aufrufen
        return {}


In [ ]:
import os
from rdflib import Graph, Namespace
import ipywidgets as widgets



shacl_output = widgets.Output()

# === SHACL-Regeln anwenden auf hochgeladene Datei ===
def apply_shacl_rules(instance_file_path: str) -> Graph:
    SH = Namespace("http://www.w3.org/ns/shacl#")
    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    shapes_dir = "Shapes"  # relative Pfadangabe im Repo

    shape_files = [
        "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
        "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
        "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
        "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
        "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
        "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
        "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
        "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
        "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
        "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
        "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl", "Y4-shapes.ttl"
    ]

    g_instance = Graph()
    g_instance.parse(instance_file_path, format="turtle")

    triples_before = len(g_instance)

    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_path}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
            with shacl_output:
                print(f"✅ Loaded SHACL shape file: {shape_file}")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error loading {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Error executing rule from {shape_file}: {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print("✅ All rules applied.")
        print(f"📊 Triples before: {triples_before}")
        print(f"📈 Triples after: {triples_after}")
        print(f"➕ Added: {triples_after - triples_before} triples")

    return g_instance


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from rdflib import Graph

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()
result_graph = None



# === Auswahlfeld: Beispiel oder Upload ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"📁 Example file selected:\n→ {instance_file_path}")
        elif change['new'] == 'upload':
            print("📤 Please upload a TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload(change):
    global instance_file_path
    if upload_widget.value:
        uploaded = upload_widget.value[0]
        file_name = uploaded['name']
        instance_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded['content'])
        output.clear_output()
        with output:
            print(f"✅ File uploaded and saved as:\n→ {file_name}")

upload_widget.observe(on_upload, names='value')

# === Continue-Button-Handler ===
def on_continue(b):
    global result_graph
    output.clear_output()

    # 🛑 Spezialfall: Upload gewählt, aber kein File hochgeladen
    if option_selector.value == "upload" and not upload_widget.value:
        with output:
            print("⚠️ Please upload a TTL file before continuing.")
        return

    # ❓ Sicherheitsabfrage: Ist ein Pfad gesetzt?
    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    # ❌ Existiert die Datei überhaupt?
    if not os.path.exists(instance_file_path):
        with output:
            print(f"❌ File not found on disk:\n→ {instance_file_path}")
        return

    try:
        if instance_file_path.endswith(".ttl") and not instance_file_path.startswith("Instance_Files/"):
            with output:
                print("🔍 SHACL rules will be applied to uploaded file.")
            result_graph = apply_shacl_rules(instance_file_path)
        else:
            g = Graph()
            g.parse(instance_file_path, format="turtle")
            result_graph = g

        with output:
            print(f"✅ RDF file successfully loaded!")
            print(f"📄 Triples in graph: {len(result_graph)}")
            print(f"🔗 Using file:\n→ {instance_file_path}")

    except Exception as e:
        with output:
            print(f"❌ Error parsing TTL file:\n→ {e}")

    # ⬇️ SHACL-Ausgabe sichtbar machen
    display(shacl_output)


continue_button.on_click(on_continue)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)

# ⬅️ Wichtig: Initiale Auswahlverarbeitung starten
on_option_change({'new': option_selector.value})

# Query-Handler
def on_query_run(b):
    global matched_systems
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if 'result_graph' not in globals() or result_graph is None:
        with query_interface_output:
            print("⚠️ RDF graph not loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ Query error: {e}")
        return

    matched_systems = []
    for row in results:
        uri = str(row.system)
        sys_id = uri.split("/")[-1]
        matched_systems.append((sys_id, uri))

    if not matched_systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = [(sys_id, uri) for sys_id, uri in matched_systems]

    with query_interface_output:
        print(f"✅ Found {len(matched_systems)} matching system(s):")
        for i, (sys_id, _) in enumerate(matched_systems):
            print(f" {i+1}: {sys_id}")

# Auswahl-Handler
def on_confirm_selection(b):
    global selected_system_id, selected_system_uri, instance_data
    
    pattern_structure = None

    selected_label = system_selector.label
    selected_uri = system_selector.value

    if not selected_uri:
        with selection_output:
            selection_output.clear_output()
            print("⚠️ No system selected.")
        return

    selected_system_id = selected_label
    selected_system_uri = selected_uri

    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_system_id}")
        print(f"🔗 URI: {selected_system_uri}")
        #print(f"🔄 Extracting instance data...")

    try:
        # Extrahiere Instanz als JSON
        extract_and_export_selected_system()

        # Lade direkt die erzeugte JSON-Datei
        json_file = f"{selected_system_id}.json"
        load_instance_json(json_file)

    except Exception as e:
        with selection_output:
            print("❌ Error trying to load and extact instance data:")
            traceback.print_exception(type(e), e, e.__traceback__)
            

In [ ]:
import ipywidgets as widgets
from rdflib.namespace import RDF
from IPython.display import display

# Outputs
query_interface_output = widgets.Output()
selection_output = widgets.Output()

# SPARQL Query-Feld
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?system
WHERE {
  ?system a swemls:System .
  ?system swemls:hasCorrespondingPattern ?pattern .
  FILTER(STRENDS(STR(?pattern), "A1"))
}
""",
    placeholder='Enter your SPARQL query here...',
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Buttons
run_query_button = widgets.Button(description="Run Query", button_style='primary')
run_query_button.on_click(on_query_run)

confirm_button = widgets.Button(description="Confirm selection", button_style='success')
confirm_button.on_click(on_confirm_selection)

# Dropdown für Systeme
system_selector = widgets.Dropdown(
    options=[],
    description="Select system:",
    layout=widgets.Layout(width='50%')
)

# Globales Ergebnis
matched_systems = []


# 🧩 Query-Komponenten anzeigen
display(
    widgets.VBox([
        query_input,
        run_query_button,
        query_interface_output,
        widgets.HBox([system_selector, confirm_button]),
        selection_output
    ])
)



In [ ]:
import json
from rdflib import Graph, Namespace, URIRef, RDF
import ipywidgets as widgets
from IPython.display import display

json_output = widgets.Output()

def extract_and_export_selected_system():
    global selected_system_id, selected_system_uri, result_graph

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology#")
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

    rdf_type = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
    ai_system_type = SWEMLS.System
    documentation_type = SWEMLS.Documentation
    paper_type = SWEMLS.Paper
    ml_component_type = SWEMLS.MachineLearningComponent
    kr_component_type = SWEMLS.KnowledgeRepresentationComponent
    data_type = SWEMLS.Data
    semantic_web_resource_type = SWEMLS.SemanticWebResource
    has_documentation = SWEMLS.hasDocumentation
    reports_on = SWEMLS.reports
    rdfs_label = RDFS.label

    g = result_graph

    # 1. System und zugehörige Paper finden
    ai_systems = [
        system for system in g.subjects(RDF.type, ai_system_type)
        if str(system).split("/")[-1] == selected_system_id
    ]

    system_to_paper = {}
    for paper in g.subjects(predicate=rdf_type, object=paper_type):
        for reported_system in g.objects(subject=paper, predicate=reports_on):
            system_id = str(reported_system).split("/")[-1]
            paper_id = str(paper).split("/")[-1]
            paper_metadata = {"id": paper_id, "metadata": {}}
            for pred, obj in g.predicate_objects(subject=paper):
                pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
                paper_metadata["metadata"].setdefault(pred_name, []).append(str(obj))
            system_to_paper.setdefault(system_id, []).append(paper_metadata)

    # 2. System extrahieren
    for system in ai_systems:
        instance_data = {
            "id": str(system).split("/")[-1],
            "type": "System",
            "metadata": {},
            "relationships": {},
            "documentation": {},
            "papers": [],
            "steps": [],
            "variables": {}
        }

        for pred, obj in g.predicate_objects(subject=system):
            pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
            if isinstance(obj, URIRef):
                instance_data["relationships"].setdefault(pred_name, []).append(str(obj).split("/")[-1])
            else:
                instance_data["metadata"][pred_name] = str(obj)

        # 3. Dokumentation extrahieren
        if "hasDocumentation" in instance_data["relationships"]:
            for doc_id in instance_data["relationships"]["hasDocumentation"]:
                doc_uri = URIRef(f"http://semantic-systems.net/swemls/{doc_id}")
                if (doc_uri, rdf_type, documentation_type) in g:
                    instance_data["documentation"]["id"] = doc_id
                    for doc_pred, doc_obj in g.predicate_objects(subject=doc_uri):
                        doc_pred_name = doc_pred.split("#")[-1] if "#" in doc_pred else doc_pred.split("/")[-1]
                        instance_data["documentation"][doc_pred_name] = str(doc_obj)

        # 4. Paper hinzufügen
        if selected_system_id in system_to_paper:
            instance_data["papers"] = system_to_paper[selected_system_id]

        # 5. Schritte (ML/KR) extrahieren
        for i in range(1, 99):
            for step_relation in [f"hasStepML{i}", f"hasStepKR{i}"]:
                if step_relation in instance_data["relationships"]:
                    for step_id in instance_data["relationships"][step_relation]:
                        step_uri = URIRef(f"http://semantic-systems.net/swemls/{step_id}")
                        step_type = "Unknown"
                        if (step_uri, rdf_type, ml_component_type) in g:
                            step_type = "Machine Learning"
                        elif (step_uri, rdf_type, kr_component_type) in g:
                            step_type = "Knowledge Representation"
                        step_data = {"id": step_id, "type": step_type, "metadata": {}}
                        for step_pred, step_obj in g.predicate_objects(subject=step_uri):
                            step_pred_name = step_pred.split("#")[-1] if "#" in step_pred else step_pred.split("/")[-1]
                            step_data["metadata"].setdefault(step_pred_name, []).append(str(step_obj))
                        instance_data["steps"].append(step_data)

        # 6. Variablen extrahieren
        for i in range(1, 99):
            for var_relation in [f"hasVariableData{i}", f"hasVariableSW{i}"]:
                if var_relation in instance_data["relationships"]:
                    for var_id in instance_data["relationships"][var_relation]:
                        var_uri = URIRef(f"http://semantic-systems.net/swemls/{var_id}")
                        label = None
                        for _, _, label_value in g.triples((var_uri, rdfs_label, None)):
                            label = str(label_value)
                            break
                        instance_data["variables"][var_relation] = {"id": var_id, "label": label}

        # 7. Speichern
        json_filename = f"{selected_system_id}.json"
        with open(json_filename, "w") as f:
            json.dump(instance_data, f, indent=4)

        with json_output:
            json_output.clear_output()
            print(f"✅ JSON successfully exported as: {json_filename}")
            print(json.dumps(instance_data, indent=2))  # Ausgabe für Überprüfung

# === Button zum Starten der Extraktion ===
#extract_button = widgets.Button(description="Extract JSON", button_style="success")
#extract_button.on_click(lambda b: extract_and_export_selected_system())

#display(extract_button, json_output)


In [ ]:
import json
from IPython.display import display
import ipywidgets as widgets

pattern_output = widgets.Output()

# === Component Mapping für automatische Generierung oder Pattern-Upload ===
import re
import json

from IPython.display import display

def run_component_mapping():
    global components, json_to_pattern_map, edges

    #display("🧩 Starte Component Mapping ...")

    # Schritt 1: Mapping von JSON zu Pattern-Kürzeln (Verwendung der ID anstelle des Labels)
    json_to_pattern_map = {}

    for step in instance_data["steps"]:
        step_key = step["id"].split(".")[-1]  # Extrahiere die ID des Schrittes
        json_to_pattern_map[step_key] = step["id"]  # Speichere die ID statt des Labels

    for var_key, var_data in instance_data["variables"].items():
        json_to_pattern_map[var_key] = var_data["id"]  # Verwende die ID der Variablen

    for var_data in instance_data["variables"].values():
        key = var_data["id"].split(".")[-1]
        if key not in json_to_pattern_map:
            json_to_pattern_map[key] = var_data["id"]

    # Schritt 2: Komponentenstruktur initialisieren
    components = {
        "ml_component": [],
        "kr_component": [],
        "data": [],
        "symbolic_data": []
    }

    for step in instance_data["steps"]:
        step_type_list = step.get("metadata", {}).get("type", [])
        step_type = step_type_list[-1] if step_type_list else ""
        if "MachineLearningComponent" in step_type and step["id"] not in components["ml_component"]:
            components["ml_component"].append(step["id"])
        elif "KnowledgeRepresentationComponent" in step_type and step["id"] not in components["kr_component"]:
            components["kr_component"].append(step["id"])

    for var_relation, var_data in instance_data["variables"].items():
        if "SW" in var_relation:
            components["symbolic_data"].append(var_data["id"])
        else:
            components["data"].append(var_data["id"])

    # Schritt 3: Fehlende Variablen im Pattern ergänzen
    for key in set(json_to_pattern_map.keys()) - set(pattern_structure["variables"].keys()):
        pattern_structure["variables"][key] = {"generated_by": []}

    # Schritt 4: Letzter Check auf fehlende Nodes
    missing_nodes = set(json_to_pattern_map.keys()) - set(pattern_structure["variables"].keys()) - set(pattern_structure["steps"].keys())
    if missing_nodes:
        display(f" ⚠️ WARNING: Nodes still missing after fix: {missing_nodes}")

    # === Edge Mapping ===
    #display("\n🔗 Starte Edge Mapping...")

    edges = []  # Initialize edges list

    # Dynamische Zuordnung spezieller Variablen
    special_mappings = {f"Data{i}": f"hasVariableData{i}" for i in range(1, 99)}
    special_mappings.update({f"SW{i}": f"hasVariableSW{i}" for i in range(1, 99)})

    # Fehlende Pattern-Variablen ergänzen
    for var_name in pattern_structure["variables"]:
        if var_name not in json_to_pattern_map:
            found_match = False

            if var_name in special_mappings:
                mapped_var = instance_data["variables"].get(special_mappings[var_name], {}).get("id")
                if mapped_var:
                    json_to_pattern_map[var_name] = mapped_var
                    found_match = True

            if not found_match:
                for var_key, var_data in instance_data["variables"].items():
                    if var_name.lower() in var_key.lower():
                        json_to_pattern_map[var_name] = var_data["id"]
                        found_match = True
                        break

            if not found_match:
                for rel_key, rel_values in instance_data["relationships"].items():
                    if isinstance(rel_values, list):
                        for value in rel_values:
                            if var_name.lower() == value.split(".")[-1].lower():
                                json_to_pattern_map[var_name] = value
                                found_match = True
                                break
                    if found_match:
                        break

            if not found_match:
                json_to_pattern_map[var_name] = var_name

    # Kanten generieren (Hier wird sichergestellt, dass alle echten IDs verwendet werden)
    for step, relations in pattern_structure["steps"].items():
        step_real_id = json_to_pattern_map.get(step, step)

        # "used"-Kanten erstellen
        for used in relations.get("uses", []):
            used_real_id = json_to_pattern_map.get(used, used)
            if used_real_id and step_real_id:
                edges.append((used_real_id, step_real_id))

        # "output"-Kanten erstellen
        for output in relations.get("outputs", []):
            output_real_id = json_to_pattern_map.get(output, output)
            if output_real_id and step_real_id:
                edges.append((step_real_id, output_real_id))

    #display(f"\n✅ Final Edge List: {edges}")

    # Prüfen auf unverbundene Knoten
    all_edge_nodes = {node for edge in edges for node in edge}
    unlinked_nodes = set(json_to_pattern_map.values()) - all_edge_nodes
    if unlinked_nodes:
        display(f"⚠️ WARNING: Nodes in mapping but not used in edges: {unlinked_nodes}")

    # Ausgabe der components-Struktur
    #display("\n🧩 Components Struktur:")
    #display(components)

import networkx as nx
from collections import defaultdict

def assign_ranks_with_limited_correction(components, edges):
    #print("🔧 Starte Topological Rank Assignment mit begrenzter Korrektur (nur bei Knoten ohne Vorgänger)...\n")

    # Schritt 1: Graph konstruieren
    G = nx.DiGraph()
    G.add_edges_from(edges)

    if not nx.is_directed_acyclic_graph(G):
        print("❌ Error: Graph contains cycle.")
        return None

    topo_order = list(nx.topological_sort(G))

    # Schritt 2: Initiale Rangzuweisung
    rank_map = {}
    for node in topo_order:
        preds = list(G.predecessors(node))

        if not preds:
            rank_map[node] = 0
        else:
            rank_map[node] = max(rank_map[p] + 1 for p in preds)

    #print("🎯 Vorläufige Ränge:")
    #for node, rank in rank_map.items():
        #print(f"  {node}: Rang {rank}")

    # Schritt 3: Korrektur nur für Knoten ohne Vorgänger
    #print("\n🔁 Starte begrenzte Shared-Input-Korrektur für Knoten ohne Vorgänger...")
    
    all_targets = defaultdict(list)
    for source, target in edges:
        all_targets[source].append(target)

    for node, targets in all_targets.items():
        if list(G.predecessors(node)):  # Skip nodes with predecessors
            continue

        candidate_ranks = []
        for target in targets:
            target_rank = rank_map.get(target, 1)
            candidate_ranks.append(target_rank - 1)

        if candidate_ranks:
            new_rank = min(candidate_ranks)
            if new_rank >= 0 and new_rank != rank_map[node]:
                #print(f"  🔁 Korrektur: {node} von Rang {rank_map[node]} → {new_rank} (basierend auf {len(candidate_ranks)} Targets)")
                rank_map[node] = new_rank

    # Schritt 4: Gruppierung nach Rängen
    grouped = defaultdict(list)
    for node, rank in rank_map.items():
        grouped[rank].append(node)

    #print("\n📊 Final zugewiesene Ränge nach Korrektur:")
    #for rank in sorted(grouped):
        #print(f"  Rang {rank}: {grouped[rank]}")

    return rank_map


def apply_positioning_algorithm():
    global positions  # wichtig, damit du später darauf zugreifen kannst

    if components and edges:
        #print("🚀 Starte Positionierungsalgorithmus...")

        rank_map = assign_ranks_with_limited_correction(components, edges)

        if rank_map:
            #print("\n📐 Rangzuweisung erfolgreich.")

            if pattern_choice in ["generate", "upload"]:
                #print("🔁 Wende Median-Heuristik an...")
                ordered_layers = apply_median_heuristic(rank_map, edges, sort_layer_0=True)

                #print("🧩 Starte Greedy-Swap...")
                optimized_layers = greedy_swap_optimization(ordered_layers, edges)

                #print("📍 Koordinaten zuweisen...")
                positions = assign_coordinates(optimized_layers)

                #print("\n📍 Initiale Koordinaten:")
                #for node, pos in positions.items():
                    #print(f"  {node}: {pos}")

                #print("🧠 Letzter Überlappungs-Fix...")
                positions = simple_overlap_correction(positions, edges)

                #print("\n📍 Final zugewiesene Koordinaten:")
                #for node, pos in positions.items():
                    #print(f"  {node}: {pos}")
                finalize_layout_parameters()  # ⬅️ hier den finalen Check aufrufen


                return rank_map

        else:
            print("⚠️ Rank assigment unsuccessful.")
            return None
    else:
        print("⚠️ Component or Edge missing")
        return None







# === Funktion zur ID-Deduplizierung ===
def deduplicate_ids(variables):
    seen_ids = {}
    updated_variables = {}

    for key, var in variables.items():
        original_id = var["id"]
        label = var.get("label", "")

        if original_id in seen_ids:
            seen_ids[original_id] += 1
            new_id = f"{original_id}_{seen_ids[original_id]}"
        else:
            seen_ids[original_id] = 1
            new_id = original_id

        updated_variables[key] = {
            "id": new_id,
            "label": label
        }

    return updated_variables

# === JSON-Datei einlesen und Variablen setzen ===
def load_instance_json(json_filename):
    global instance_data, pattern, pattern_structure  # 🆕 pattern_structure ergänzt

    try:
        with open(json_filename, "r") as json_file:
            extracted_data = json.load(json_file)
            instance_data = extracted_data

        # 🧹 Wichtig: Alte pattern_structure zurücksetzen
        pattern_structure = None

        # System-Label und Pattern-URI extrahieren
        system_label = instance_data.get("metadata", {}).get("label", "no label")
        raw_pattern_uri = instance_data.get("relationships", {}).get("hasCorrespondingPattern", [None])[0]

        # Sicher extrahieren – falls kein Pattern vorhanden, als "no pattern" setzen
        if raw_pattern_uri:
            pattern = raw_pattern_uri.split(".")[-1] if "Pattern." in raw_pattern_uri else raw_pattern_uri.split("/")[-1]
        else:
            pattern = "no pattern"

        # IDs deduplizieren
        instance_data["variables"] = deduplicate_ids(instance_data.get("variables", {}))

        # Danach weitere Pattern-Entscheidung anzeigen
        handle_pattern_selection()

    except FileNotFoundError:
        with pattern_output:
            pattern_output.clear_output()
            print("❌ JSON file not found! Please extract an instance first.")
    except Exception as e:
        with pattern_output:
            pattern_output.clear_output()
            print(f"❌ Error loading JSON:\n→ {e}")


# === Button zur Ausführung ===
#load_button = widgets.Button(description="Load Extracted JSON", button_style="primary")

#def on_load_click(b):
 #   try:
  #      json_file = f"{selected_system_id}.json"
   #     load_instance_json(json_file)
    #except NameError:
      #  with pattern_output:
     #       pattern_output.clear_output()
       #     print("❌ No system selected. Please extract a system first.")

#load_button.on_click(on_load_click)


#display(load_button, pattern_output)


In [ ]:
import os
import json
import re
from rdflib import Graph, Namespace, RDF
import ipywidgets as widgets
from IPython.display import display, clear_output

# === Globals for reuse ===
pattern_structure = None
pattern_choice = None
pattern_graph = None
pattern = None

# === Variables from previous steps assumed ===
# instance_data (dict), pattern (str or "no pattern")

# === Output containers ===
pattern_output = widgets.Output()
pattern_upload_output = widgets.Output()

download_button = widgets.Button(
    description="⬇️ Download Template XML",
    button_style="info",
    icon="download"
)
download_output = widgets.Output()


pattern_continue_button = widgets.Button(
    description="Continue with uploaded pattern",
    button_style="success"
)

# === Upload widget (only shown when needed) ===
pattern_upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload Pattern File'
)

# === Pattern decision: Template or Generate ===
pattern_choice_selector = widgets.ToggleButtons(
    options=[
        ("Use template", "template"),
        ("Generate automatically", "generate")
    ],
    description="Select Pattern Option:",
    style={'description_width': 'initial'}
)


def generate_template_from_instance():
    global pattern, instance_data, pattern_structure

    with download_output:
        download_output.clear_output()
        try:
            #print("🛠 generate_template_from_instance() aufgerufen")
            #print(f"🔍 Aktuelles Pattern: {pattern}")
            #print(f"📦 instance_data vorhanden: {'instance_data' in globals()}")
            #print(f"📦 pattern_structure vorhanden: {pattern_structure is not None}")

            if not pattern or pattern in ("no pattern", "None", None):
                print("⚠️ No Pattern available!")
                return

            # Falls pattern_structure fehlt: nachladen
            if not pattern_structure:
                pattern_file_path = f"Patterns/{pattern}-pattern.ttl"
                if os.path.exists(pattern_file_path):
                    #print(f"📄 Lade Pattern-Datei: {pattern_file_path}")
                    pattern_structure = extract_pattern_structure_from_file(pattern_file_path)
                else:
                    print(f"❌ Pattern-file missing: {pattern_file_path}")
                    return

            if not pattern_structure or not isinstance(pattern_structure, dict):
                print("❌ Pattern file missing or empty pattern_structure")
                return

            #print(f"📊 pattern_structure preview:")
            #print(json.dumps(pattern_structure, indent=2))

            # Template laden
            template_path = f"Templates/{pattern}.xml"
            if not os.path.exists(template_path):
                print(f"❌ Template-file missing: {template_path}")
                return

            with open(template_path, "r", encoding="utf-8") as f:
                xml_template = f.read()
                #print(f"✅ Template geladen ({len(xml_template)} Zeichen)")

            updated_xml = replace_pattern_placeholders(xml_template, instance_data, pattern_structure)

            if not updated_xml.strip():
                print("❌ Generated XML ist empty")
                return

            instance_id = instance_data.get("id", "unknown")
            filename = f"Updated_{instance_id}_Workflow.xml"

            with open(filename, "w", encoding="utf-8") as f:
                f.write(updated_xml)
                print(f"✅ XML saved: {filename}")

            b64 = base64.b64encode(updated_xml.encode()).decode()
            href = f'data:application/xml;base64,{b64}'
            display(HTML(f'<a download="{filename}" href="{href}" target="_blank">⬇️ Click here to download the XML file</a>'))

            # Starte Merge
            finalize_and_merge_xml(filename)

        except Exception as e:
            print("❌ Error while generating template")
            traceback.print_exception(type(e), e, e.__traceback__)


def handle_pattern_selection():
    global pattern, pattern_choice, pattern_structure

    pattern_output.clear_output()
    pattern_upload_output.clear_output()

    try:
        with pattern_output:
            print(f"🔍 pattern: {pattern}")

            if pattern is None or pattern == "no pattern":
                print("📂 No pattern linked to system. Please upload a pattern TTL file.")
                display(pattern_upload_widget)
                display(pattern_continue_button)
                display(pattern_upload_output)
            else:
                pattern_file_path = f"Patterns/{pattern}-pattern.ttl"
                if not os.path.exists(pattern_file_path):
                    print(f"⚠️ Pattern-file for {pattern} not found.")
                    print("📤 Please upload valid pattern file.")
                    display(pattern_upload_widget)
                    display(pattern_continue_button)
                    display(pattern_upload_output)
                else:
                    print(f"🧩 Pattern detected: {pattern}")
                    print("How would you like to proceed?")
                    display(pattern_choice_selector)

                    # Nur anzeigen, wenn nicht bereits sichtbar
                    if pattern_decision_continue_button.layout.display != "inline-block":
                        pattern_decision_continue_button.layout.display = "inline-block"

    except Exception as e:
        show_exception(e)




        
def on_pattern_continue(b):
    global pattern_structure, pattern_choice, pattern
    pattern_structure = None

    with pattern_output:
        pattern_output.clear_output()
        #print("🟢 [DEBUG] Button clicked!")

        if not pattern_upload_widget.value:
            print("⚠️ No file uploaded.")
            return

        upload_info = pattern_upload_widget.value[0]
        file_name = upload_info['name']
        file_content = upload_info['content']

        with open(file_name, "wb") as f:
            f.write(file_content)

        #print(f"🟢 [DEBUG] File '{file_name}' written to disk.")

        try:
            pattern_structure = extract_pattern_structure_from_file(file_name)
            #print("✅ Pattern structure parsed.")

            pattern_choice = "upload"
            pattern = "custom"

            # Sofort generieren und mergen!
            run_component_mapping()
            apply_positioning_algorithm()

            instance_id = instance_data.get("id", "unknown")
            workflow_path = f"Generated_{instance_id}_Workflow.drawio"

            xml_output = generate_drawio_xml(positions, components, edges, label_map)

            with open(workflow_path, "w", encoding="utf-8") as f:
                f.write(xml_output)

            # Download-Link erzeugen
            b64 = base64.b64encode(xml_output.encode()).decode()
            href = f'data:application/xml;base64,{b64}'
            #display(HTML(f'<a download="{workflow_path}" href="{href}" target="_blank">⬇️ Click here to download the XML file</a>'))

            #print(f"📥 File ready: {workflow_path}")

            finalize_and_merge_xml(workflow_path)

            # 🔁 Kein weiterer Button nötig!
            pattern_decision_continue_button.layout.display = "none"

        except Exception as e:
            print(f"❌ Error parsing pattern file or generating XML: {e}")



# Am Ende registrieren:
pattern_continue_button.on_click(on_pattern_continue)


# === Extract logic from TTL file ===
def extract_pattern_structure_from_file(file_path):
    g = Graph()
    g.parse(file_path, format="turtle")

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology/")
    RES = Namespace("http://semantic-systems.net/swemls/")

    def clean_uri(uri):
        label = uri.split("/")[-1].split("#")[-1]
        label = re.sub(r'Pattern\.[A-Za-z0-9]+\.', '', label)
        label = re.sub(r'^[A-Za-z0-9]+\.', '', label)
        return label

    structure = {"steps": {}, "variables": {}}

    for step in g.subjects(RDF.type, SWEMLS.WorkflowTemplateProcessML):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "ML", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, SWEMLS.WorkflowTemplateProcessKR):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "KR", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, RES.WorkflowTemplateProcessML):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "ML", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, RES.WorkflowTemplateProcessKR):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "KR", "uses": inputs, "outputs": []}

    for var in g.subjects(RDF.type, SWEMLS.TemplateArtifactData):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, RES.TemplateArtifactData):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, SWEMLS.TemplateArtifactSW):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, RES.TemplateArtifactSW):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    return structure

# === Trigger on pattern upload ===
def on_pattern_upload(change):
    global pattern_structure

    if not pattern_upload_widget.value:
        return

    uploaded = next(iter(pattern_upload_widget.value.items()))  # (filename, fileinfo)
    file_name = uploaded[0]
    file_content = uploaded[1]['content']

    with open(file_name, "wb") as f:
        f.write(file_content)

    with pattern_output:
        clear_output()
        print(f"✅ Pattern file uploaded: {file_name}")

    try:
        pattern_structure = extract_pattern_structure_from_file(file_name)
        with pattern_output:
            print("✅ Extracted pattern structure:")
            print(json.dumps(pattern_structure, indent=4))
    except Exception as e:
        with pattern_output:
            print(f"❌ Error parsing pattern file: {e}")




pattern_upload_widget.observe(on_pattern_upload, names='value')



def on_pattern_choice(change):
    global pattern_choice
    pattern_choice = change['new']
    # Kein sofortiger Code-Start!


pattern_choice_selector.observe(on_pattern_choice, names='value')




# Danach Anzeige und Auswahl starten
main_pattern_box = widgets.VBox([
    pattern_output,
    pattern_upload_output,
    download_output  # 🆕 Zeigt den Download-Link nach Template-Erstellung
])


# Button-Handler aktivieren ⬇️
pattern_continue_button.on_click(on_pattern_continue)

# Danach Anzeige und Auswahl starten
handle_pattern_selection()
display(main_pattern_box)


In [ ]:
import sys
import traceback
import os
import base64
import ipywidgets as widgets
from IPython.display import display, HTML
import re
import json
import xml.etree.ElementTree as ET

# === Globale Ausgabe-Widgets ===
download_output = widgets.Output()

# === Funktion zur Fehlerausgabe ===
def show_exception(e):
    print("❌ Exception caught:")
    traceback.print_exception(type(e), e, e.__traceback__, file=sys.stdout)

# === Funktion zum Extrahieren des Labels aus einem Schritt ===
def extract_label_short(step):
    labels = step.get("metadata", {}).get("label", [])
    if labels:
        match = re.search(r"\((.*?)\)", labels[0])
        if match:
            return match.group(1)
    return step["id"].split(".")[-1]

# === Funktion zum Ersetzen der Platzhalter im XML ===
def replace_pattern_placeholders(xml_string, instance_data, pattern_structure):
    #with download_output:
     #   print("\U0001f527 replace_pattern_placeholders() aufgerufen")
      #  print(f"\U0001f527 XML Länge: {len(xml_string)}")
       # print(f"\U0001f527 Instance data keys: {list(instance_data.keys())}")
        #print(f"\U0001f527 Pattern structure keys: {list(pattern_structure.keys())}")

    tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()

    replacements = {}

    #with download_output:
        #print("\U0001f501 Schritte werden ersetzt...")
    for step in instance_data.get("steps", []):
        step_key = step["id"].split(".")[-1]
        step_label = extract_label_short(step)
        replacements[step_key] = step_label

    #with download_output:
        #print("\U0001f501 Variablen werden ersetzt...")
    for pattern_var in pattern_structure.get("variables", {}):
        for var_key, var_data in instance_data.get("variables", {}).items():
            if pattern_var.lower() in var_key.lower():
                inst_id = var_data["id"].replace("Resource.", "").replace("Custom.", "")
                replacements[pattern_var] = inst_id
                break

    #with download_output:
        #print("\n\U0001f9e9 Mapping for replacements:")
        #for k, v in replacements.items():
           # print(f"  {k} → {v}")

    for element in root.iter("mxCell"):
        if 'value' in element.attrib:
            value = element.attrib['value']
            for placeholder, real_value in replacements.items():
                if placeholder in value:
                    replacement = real_value.replace("_", " ") if real_value.strip() else "Missing"
                    style = element.attrib.get("style", "")
                    length = len(replacement)
                    if length > 24:
                        style += ";fontSize=5"
                    elif length > 20:
                        style += ";fontSize=7"
                    elif length > 17:
                        style += ";fontSize=9"
                    element.attrib['style'] = style
                    element.attrib['value'] = value.replace(placeholder, replacement)

    return ET.tostring(root, encoding='utf-8', method='xml').decode()

# === Template generieren und Base64-Download-Link anzeigen ===
def generate_template_from_instance():
    global pattern, instance_data, pattern_structure

    with download_output:
        download_output.clear_output()
        try:
            #print("🛠 generate_template_from_instance() aufgerufen")
            #print(f"🔍 Aktuelles Pattern: {pattern}")
            #print(f"📦 instance_data vorhanden: {'instance_data' in globals()}")
            #print(f"📦 pattern_structure vorhanden: {pattern_structure is not None}")

            if not pattern or pattern in ("no pattern", "None", None):
                print("⚠️ No valid pattern")
                return

            # Falls pattern_structure fehlt: nachladen
            if not pattern_structure:
                pattern_file_path = f"Patterns/{pattern}-pattern.ttl"
                if os.path.exists(pattern_file_path):
                    #print(f"📄 Lade Pattern-Datei: {pattern_file_path}")
                    pattern_structure = extract_pattern_structure_from_file(pattern_file_path)
                else:
                    print(f"❌ Pattern-file missing: {pattern_file_path}")
                    return

            if not pattern_structure or not isinstance(pattern_structure, dict):
                print("❌ Invalid or empty pattern_structure")
                return

            #print(f"📊 pattern_structure preview:")
            #print(json.dumps(pattern_structure, indent=2))

            # Template laden
            template_path = f"Templates/{pattern}.xml"
            if not os.path.exists(template_path):
                print(f"❌ Template file not found: {template_path}")
                return

            with open(template_path, "r", encoding="utf-8") as f:
                xml_template = f.read()
                #print(f"✅ Template-Datei geladen mit Länge {len(xml_template)}")

            updated_xml = replace_pattern_placeholders(xml_template, instance_data, pattern_structure)

            if not updated_xml.strip():
                print("❌ Generated XML is empty")
                return

            instance_id = instance_data.get("id", "unknown")
            custom_filename = f"Updated_{instance_id}_Workflow.xml"

            # 1. Speichern für den Download
            with open(custom_filename, "w", encoding="utf-8") as f:
                f.write(updated_xml)
                #print(f"✅ XML gespeichert unter: {custom_filename}")

            # 2. Speichern unter fixed Name für den Merge
            with open("Generated_Workflow.drawio", "w", encoding="utf-8") as f:
                f.write(updated_xml)
                #print("✅ XML auch unter Generated_Workflow.drawio gespeichert (für Merge)")

            # Download-Link erzeugen
            b64 = base64.b64encode(updated_xml.encode()).decode()
            href = f'data:application/xml;base64,{b64}'
            display(HTML(f'<a download="{custom_filename}" href="{href}" target="_blank">⬇️ Click here to download the XML file</a>'))

            print(f"📥 File ready: {custom_filename}")

            # 🧩 Merge starten mit fixer Datei
            finalize_and_merge_xml("Generated_Workflow.drawio")

        except Exception as e:
            print("❌ Error while generating template")
            traceback.print_exception(type(e), e, e.__traceback__)

# 🆕 Entscheidung durchführen Button
pattern_decision_continue_button = widgets.Button(
    description="Continue with selected option",
    button_style="primary",
    icon="check"
)


def on_pattern_decision_continue(b):
    global pattern_choice, pattern_structure, pattern

    # Ersten Wert übernehmen
    pattern_choice = pattern_choice_selector.value

    # ⛑️ Fallback: Wenn Upload aktiv war, überschreibe Auswahl
    if pattern_upload_widget.value and pattern == "custom":
        pattern_choice = "upload"

    with download_output:
        download_output.clear_output()
        print("➡️ Continue decision clicked")
        print(f"🧭 Selected option (final): {pattern_choice}")

        try:
            instance_id = instance_data.get("id", "unknown")
            workflow_path = f"Generated_{instance_id}_Workflow.drawio"

            if pattern_choice == "template":
                # Sicherstellen, dass pattern_structure geladen ist
                if not pattern_structure:
                    file_path = f"Patterns/{pattern}-pattern.ttl"
                    if os.path.exists(file_path):
                        print(f"📂 Reloading pattern file: {file_path}")
                        pattern_structure = extract_pattern_structure_from_file(file_path)
                    else:
                        print(f"❌ Pattern-file missing or unable to load: {file_path}")
                        return

                generate_template_from_instance()
                template_path = f"Updated_{instance_id}_Workflow.xml"
                finalize_and_merge_xml(template_path)

            elif pattern_choice in ["generate", "upload"]:
                # ✅ NEU: Falls generate gewählt wurde, aber pattern_structure noch fehlt → nachladen
                if not pattern_structure and pattern not in ("no pattern", None):
                    file_path = f"Patterns/{pattern}-pattern.ttl"
                    if os.path.exists(file_path):
                        print(f"📂 Auto-loading pattern file: {file_path}")
                        pattern_structure = extract_pattern_structure_from_file(file_path)
                    else:
                        print(f"❌ Pattern-file missing: {file_path}")
                        return

                run_component_mapping()
                apply_positioning_algorithm()

                xml_output = generate_drawio_xml(positions, components, edges, label_map)

                with open(workflow_path, "w", encoding="utf-8") as f:
                    f.write(xml_output)

                finalize_and_merge_xml(workflow_path)

            else:
                print("⚠️ No valid option selected")

        except Exception as e:
            show_exception(e)

    pattern_decision_continue_button.layout.display = "none"



pattern_decision_continue_button.on_click(on_pattern_decision_continue)

# 🆕 Anzeige des Buttons und Ausgabecontainers
display(pattern_decision_continue_button, download_output)

# ✅ Anzeigen im Notebook
#display(component_mapping_output, edge_mapping_output)
